![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+ML%2FSolution+Prototypes%2Fdocument-processing%2F7-agents%2Fdocument_agent&file=deploy-vertex-ai-agent-engine.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/deploy-vertex-ai-agent-engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520ML%2FSolution%2520Prototypes%2Fdocument-processing%2F7-agents%2Fdocument_agent%2Fdeploy-vertex-ai-agent-engine.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/deploy-vertex-ai-agent-engine.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/deploy-vertex-ai-agent-engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Deploy Agent To Vertex AI Agent Engine

After building and testing an agent built with ADK the next step is deployment for use in production workflows and applications.  The notebook workflow shows the steps for deploying the agent built in the product on [Vertex AI Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview).  This service includes:
- [Development](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/develop/adk) tooling to test the agent locally first
- [Deployment](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/deploy) tooling to create the agent instance on Vertex AI Agent Engine
- [Agent Use](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/adk) SDK directly within the Vetex AI SDK
- [Mangagement](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/overview) services for access control, tracing, logging, and monitoring of deployed agents
- and services like [Agent Evaluation](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/evaluate) and more!

While Vertex AI Agent Engine is a service for many agent building frameworks (ADK, LangChain, LangGraph, AG2, LlamaIndex, Custom, ...) it is also directly integrated with Google ADK, the framework used in this project.  [ADK Deployment](https://google.github.io/adk-docs/deploy/) options include [Vertex AI Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/) as well as [Cloud Run](https://google.github.io/adk-docs/deploy/cloud-run/) and [custom infrastructure](https://google.github.io/adk-docs/deploy/gke/) deploymetns.

**Notes:**
- This notebook expects the current working directory to contain the agents folder. In other words, `agent.py` is at `./document_agent/agent.py`.

Package Imports:

In [1]:
import os, sys

import dotenv
import requests

from vertexai.preview import reasoning_engines
from vertexai import agent_engines
import vertexai

Verify working directory:

In [2]:
os.getcwd()

'/home/statmike/repos/vertex-ai-mlops/Applied ML/Solution Prototypes/document-processing/7-agents/document_agent'

Review the local files, expecting the `document_agent` folder to be here:

In [3]:
os.listdir('.')

['document_agent', 'apps', 'deploy-vertex-ai-agent-engine.ipynb']

Verify the current Python version and executable location (the `venv` setup by this project):

In [4]:
sys.version, sys.executable

('3.13.3 (main, Apr 14 2025, 14:06:24) [GCC 12.2.0]',
 '/home/statmike/repos/vertex-ai-mlops/Applied ML/Solution Prototypes/document-processing/7-agents/.venv/bin/python')

Setup Clients:

In [5]:
# Load environment variables set with agent (./document_agent/.env)
dotenv.load_dotenv(dotenv_path = os.path.join('document_agent', '.env'))

# Vertex AI Initializaiton:
vertexai.init(
    project = os.getenv('GOOGLE_CLOUD_PROJECT'),
    location = os.getenv('GOOGLE_CLOUD_LOCATION'),
    staging_bucket = f"gs://{os.getenv('GOOGLE_CLOUD_STORAGE_BUCKET')}"
)

---
## Local Testing: Prepare Agent for Vertex AI Agent Engine

Before deploying the ADK agent to Vertex AI Agent Engine it can first be tested locally with the SDK.

In [6]:
from document_agent.agent import root_agent

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [7]:
session = app.create_session(user_id = 'u_123')
session

Session(id='cc428561-8e3c-4e01-829c-33924d731ca7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749316598.6712692)

In [8]:
app.list_sessions(user_id = 'u_123')

ListSessionsResponse(sessions=[Session(id='cc428561-8e3c-4e01-829c-33924d731ca7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749316598.6712692)])

In [9]:
session = app.get_session(user_id="u_123", session_id=session.id)
session

Session(id='cc428561-8e3c-4e01-829c-33924d731ca7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749316598.6712692)

In [ ]:
for event in app.stream_query(
    user_id = 'u_123',
    session_id = session.id,
    message = 'What do you do?'
):
    print(event)

In [12]:
for event in app.stream_query(
    user_id = 'u_123',
    session_id = session.id,
    message = 'use this file: gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf'
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-7c70745d-1670-473b-82c8-fd234e8ddc92', 'args': {'gcs_file_path': 'applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf', 'gcs_bucket': 'statmike-mlops-349915'}, 'name': 'get_gcs_file'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 56, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 56}], 'prompt_token_count': 3583, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 3583}], 'total_token_count': 3639, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-989f09b7-7f2b-4bf8-afa6-8526eb7ce9cb', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'F7Y2oybx', 'timestamp': 1749316611.116951}
{'content': {'parts': [{'function_response': {'id': 'adk-7c70745d-1670-473b-82c8-fd234e8ddc92', 'n

In [13]:
event

{'content': {'parts': [{'text': 'Okay, I have loaded the file from GCS. The artifact key for this document is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`.\n'}],
  'role': 'model'},
 'usage_metadata': {'candidates_token_count': 68,
  'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
    'token_count': 68}],
  'prompt_token_count': 3751,
  'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
    'token_count': 3751}],
  'total_token_count': 3819,
  'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>},
 'invocation_id': 'e-989f09b7-7f2b-4bf8-afa6-8526eb7ce9cb',
 'author': 'document_agent',
 'actions': {'state_delta': {},
  'artifact_delta': {},
  'requested_auth_configs': {}},
 'id': 'aT0ueQYB',
 'timestamp': 1749316614.02341}

In [14]:
app.get_session(user_id = 'u_123', session_id = session.id)

Session(id='cc428561-8e3c-4e01-829c-33924d731ca7', app_name='default-app-name', user_id='u_123', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='What do you do?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-b6ed94f8-4582-41ef-9d39-c306a1ca024d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='bzp9rqYP', timestamp=1749316602.632191), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executa

In [15]:
for event in app.get_session(user_id = 'u_123', session_id = session.id).events:
    print(event)

content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='What do you do?')], role='user') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None invocation_id='e-b6ed94f8-4582-41ef-9d39-c306a1ca024d' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=None branch=None id='bzp9rqYP' timestamp=1749316602.632191
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='I am the primary agent that processes document related requests. I can load doc

In [16]:
app.delete_session(user_id = 'u_123', session_id = session.id)

In [17]:
app.list_sessions(user_id = 'u_123')

ListSessionsResponse(sessions=[])

more
- create session with specified id

---
## Deploy To Vertex AI Agent Engine

Now that the ADK Agent has been tested locally it can be deployed to Vertex AI Agent Engine.

In [18]:
remote_app = agent_engines.create(
    agent_engine = root_agent,
    requirements = "../requirements.txt",
    extra_packages = ["./document_agent"],
    gcs_dir_name = 'applied-ml-solution-prototypes/document-processing/agent_engine/staging',
    display_name = root_agent.name,
    description = root_agent.description,
    #env_vars = 
)

Deploying google.adk.agents.Agent as an application.
Reading requirements from requirements='../requirements.txt'
Read the following lines: ['absl-py==2.3.0', 'aiofiles==24.1.0', 'altair==5.5.0', 'annotated-types==0.7.0', 'anyio==4.9.0', 'asttokens==3.0.0', 'attrs==25.3.0', 'audioop-lts==0.2.1', 'Authlib==1.5.2', 'blinker==1.9.0', 'cachetools==5.5.2', 'certifi==2025.4.26', 'cffi==1.17.1', 'charset-normalizer==3.4.1', 'click==8.1.8', 'cloudpickle==3.1.1', 'comm==0.2.2', 'cryptography==44.0.2', 'db-dtypes==1.4.3', 'debugpy==1.8.14', 'decorator==5.2.1', 'deepdiff==6.7.1', 'Deprecated==1.2.18', 'docstring_parser==0.16', 'executing==2.2.0', 'fastapi==0.115.12', 'ffmpy==0.5.0', 'filelock==3.18.0', 'Flask==3.1.1', 'fsspec==2025.5.1', 'gitdb==4.0.12', 'GitPython==3.1.44', 'google-adk==1.2.1', 'google-api-core==2.24.2', 'google-api-python-client==2.169.0', 'google-auth==2.39.0', 'google-auth-httplib2==0.2.0', 'google-cloud-aiplatform==1.96.0', 'google-cloud-appengine-logging==1.6.1', 'google-cl

In [183]:
remote_app.resource_name

'projects/1026793852137/locations/us-central1/reasoningEngines/1988805428414251008'

In [7]:
#remote_app = agent_engines.get(resource_name = 'projects/1026793852137/locations/us-central1/reasoningEngines/1988805428414251008')

In [185]:
dotenv.set_key(
    './apps/.env', 'ADK_DEPLOY_RESOURCE_ID', remote_app.resource_name
)

(True,
 'ADK_DEPLOY_RESOURCE_ID',
 'projects/1026793852137/locations/us-central1/reasoningEngines/1988805428414251008')

In [186]:
remote_session = remote_app.create_session(user_id = 'u_123')
remote_session

{'id': '2177099543449960448',
 'state': {},
 'events': [],
 'lastUpdateTime': 1749406351.879862,
 'appName': '1988805428414251008',
 'userId': 'u_123'}

In [187]:
remote_app.list_sessions(user_id = 'u_123')

{'sessions': [{'id': '2177099543449960448',
   'lastUpdateTime': 1749406351.879862,
   'state': {},
   'events': [],
   'appName': '1988805428414251008',
   'userId': 'u_123'}]}

In [188]:
remote_session = remote_app.get_session(user_id = "u_123", session_id = remote_session['id'])
remote_session

{'id': '2177099543449960448',
 'lastUpdateTime': 1749406351.879862,
 'state': {},
 'events': [],
 'appName': '1988805428414251008',
 'userId': 'u_123'}

In [189]:
import json
from google import genai

message = genai.types.Content(
    role = 'user',
    parts = [genai.types.Part.from_text(text = 'What do you do?')]    
)

for event in remote_app.stream_query(
    user_id = 'u_123',
    session_id = remote_session['id'],
    #message = message # DOES NOT WORK WITH GENAI OBJECTS 
    #message = json.dumps([{'text' : 'What do you do?'}]) # NO ERROR BUT IT JUST PASSES JSON TEXT AS TEXT PART
    message = 'What do you do?'
):
    print(event)

{'content': {'parts': [{'text': 'I am the primary agent that processes document related requests. I can load documents, classify them, extract data, and compare them to vendor templates. To start, please upload a PDF/PNG file or provide the GCS URI (bucket and path) for the document you want to process.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 59, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 59}], 'prompt_token_count': 3474, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3474}], 'total_token_count': 3533, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-677976b1-9315-4f8e-96c0-d31af507916e', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'y7AIIbNE', 'timestamp': 1749406356.888147}


In [190]:
async for event in remote_app.async_stream_query(
    user_id = 'u_123',
    session_id = remote_session['id'],
    message = 'What do you do best?'
):
    print(event)

{'content': {'parts': [{'text': 'I am designed to orchestrate document processing workflows. This means I am best at handling the overall process of taking a document, preparing it using various tools, and then dispatching it to specialized sub-agents for tasks like in-depth analysis, comparison, or answering specific questions about the content. I ensure all the necessary steps are completed in the correct order and that the results are presented to you clearly. I am also capable of remembering key information, such as artifact keys and classification results, to be used in subsequent steps.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 106, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 106}], 'prompt_token_count': 3539, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3539}], 'total_token_count': 3645, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-c488389d-2afb-46cf-8b12-b8e7d1e6599a', 'author': 'document_agent', 'actions': 

In [191]:
type(event)

dict

In [192]:
stream = remote_app.stream_query(
    user_id = 'u_123',
    session_id = remote_session['id'],
    message = 'What do you do first?'
)
type(stream)

generator

In [193]:
for event in stream:
    print(event)

{'content': {'parts': [{'text': 'The first thing I need to do is obtain the document you want to process. Please upload a PDF/PNG file or provide the GCS URI (bucket and path) for the document.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 39, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 39}], 'prompt_token_count': 3651, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3651}], 'total_token_count': 3690, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-b9397c1b-e387-4f51-a2d8-6430418d82e9', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '50111ldt', 'timestamp': 1749406364.031439}


In [194]:
event

{'content': {'parts': [{'text': 'The first thing I need to do is obtain the document you want to process. Please upload a PDF/PNG file or provide the GCS URI (bucket and path) for the document.\n'}],
  'role': 'model'},
 'usage_metadata': {'candidates_token_count': 39,
  'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 39}],
  'prompt_token_count': 3651,
  'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3651}],
  'total_token_count': 3690,
  'traffic_type': 'ON_DEMAND'},
 'invocation_id': 'e-b9397c1b-e387-4f51-a2d8-6430418d82e9',
 'author': 'document_agent',
 'actions': {'state_delta': {},
  'artifact_delta': {},
  'requested_auth_configs': {}},
 'id': '50111ldt',
 'timestamp': 1749406364.031439}

In [195]:
event['content']['parts'][-1]['text']

'The first thing I need to do is obtain the document you want to process. Please upload a PDF/PNG file or provide the GCS URI (bucket and path) for the document.\n'

In [196]:
for event in remote_app.stream_query(
    user_id = 'u_123',
    session_id = remote_session['id'],
    message = 'use this file: gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf'
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-ec674964-45f1-459b-8766-6a92b9fedc15', 'args': {'gcs_file_path': 'applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf', 'gcs_bucket': 'statmike-mlops-349915'}, 'name': 'get_gcs_file'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 56, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 56}], 'prompt_token_count': 3740, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3740}], 'total_token_count': 3796, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-1b95e891-dbb1-4bf9-bd7a-0f12e62241b6', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': '08qIaBFB', 'timestamp': 1749406369.072305}
{'content': {'parts': [{'function_response': {'id': 'adk-ec674964-45f1-459b-8766-6a92b9fedc15', 'name': 'get_gcs_file', 'response': {'result': 'The file vendor_2_invoice_

In [197]:
type(event), event

(dict,
 {'content': {'parts': [{'text': 'Okay, I have loaded the file from GCS. The artifact key for this document is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`.\n'}],
   'role': 'model'},
  'usage_metadata': {'candidates_token_count': 68,
   'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 68}],
   'prompt_token_count': 3908,
   'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3908}],
   'total_token_count': 3976,
   'traffic_type': 'ON_DEMAND'},
  'invocation_id': 'e-1b95e891-dbb1-4bf9-bd7a-0f12e62241b6',
  'author': 'document_agent',
  'actions': {'state_delta': {},
   'artifact_delta': {},
   'requested_auth_configs': {}},
  'id': '7FfH7ObF',
  'timestamp': 1749406370.835172})

### Use CURL:

In [198]:
URL = f"https://{os.getenv('GOOGLE_CLOUD_LOCATION')}-aiplatform.googleapis.com/v1/{remote_app.resource_name}"
URL

'https://us-central1-aiplatform.googleapis.com/v1/projects/1026793852137/locations/us-central1/reasoningEngines/1988805428414251008'

In [204]:
payload = dict(
    class_method = 'get_session',
    input = dict(
        user_id = 'u_123',
        session_id = remote_session['id']
    )
)
payload

{'class_method': 'get_session',
 'input': {'user_id': 'u_123', 'session_id': '2177099543449960448'}}

In [205]:
import google.auth

In [206]:
credentials, _ = google.auth.default()
auth_req = google.auth.transport.requests.Request()
credentials.refresh(auth_req)
headers = {'Authorization': f'Bearer {credentials.token}'}
response = requests.post(URL+':query', headers = headers, json = payload)    
response.raise_for_status()

In [207]:
response.json()

{'output': {'userId': 'u_123',
  'appName': '1988805428414251008',
  'events': [{'timestamp': 1749406356.673932,
    'interrupted': None,
    'id': '1038108851167035392',
    'partial': None,
    'usageMetadata': None,
    'errorCode': None,
    'author': 'user',
    'customMetadata': None,
    'actions': {'transferToAgent': None,
     'requestedAuthConfigs': {},
     'artifactDelta': {},
     'skipSummarization': None,
     'escalate': None,
     'stateDelta': {}},
    'longRunningToolIds': None,
    'errorMessage': None,
    'invocationId': 'e-677976b1-9315-4f8e-96c0-d31af507916e',
    'turnComplete': None,
    'groundingMetadata': None,
    'content': {'role': 'user',
     'parts': [{'videoMetadata': None,
       'text': 'What do you do?',
       'inlineData': None,
       'executableCode': None,
       'thoughtSignature': None,
       'thought': None,
       'fileData': None,
       'functionResponse': None,
       'codeExecutionResult': None,
       'functionCall': None}]},
    'b

In [216]:
payload = dict(
    class_method = 'stream_query',
    input = dict(
        user_id = 'u_123',
        session_id = remote_session['id'],
        message = 'Do you do data extraction?'
    )
)
credentials.refresh(auth_req)
headers = {'Authorization': f'Bearer {credentials.token}'}
response = requests.post(URL+':streamQuery?alt=sse', headers = headers, json = payload, stream = True)    
response.raise_for_status()

In [217]:
response.json()

{'content': {'parts': [{'text': 'Yes, I can perform data extraction on the document. Would you like me to proceed with extracting the data from the document I just loaded with artifact key `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`?\n'}],
  'role': 'model'},
 'usage_metadata': {'candidates_token_count': 81,
  'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 81}],
  'prompt_token_count': 4012,
  'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 4012}],
  'total_token_count': 4093,
  'traffic_type': 'ON_DEMAND'},
 'invocation_id': 'e-7ab1118e-7699-464e-a625-f2ceedf35927',
 'author': 'document_agent',
 'actions': {'state_delta': {},
  'artifact_delta': {},
  'requested_auth_configs': {}},
 'id': '38r65IYA',
 'timestamp': 1749415164.650748}

In [224]:
for line in response.iter_lines(decode_unicode=True):
    print(json.loads(line).get('content').get('parts')[0].get('text'))

Yes, I can perform data extraction on the document. Would you like me to proceed with extracting the data from the document I just loaded with artifact key `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`?



### Back To SDK

In [225]:
remote_app.get_session(user_id = 'u_123', session_id = remote_session['id'])

{'id': '2177099543449960448',
 'state': {},
 'lastUpdateTime': 1749415165.554814,
 'events': [{'interrupted': None,
   'invocationId': 'e-677976b1-9315-4f8e-96c0-d31af507916e',
   'partial': None,
   'branch': None,
   'id': '1038108851167035392',
   'turnComplete': None,
   'usageMetadata': None,
   'content': {'role': 'user',
    'parts': [{'fileData': None,
      'thought': None,
      'codeExecutionResult': None,
      'text': 'What do you do?',
      'inlineData': None,
      'videoMetadata': None,
      'thoughtSignature': None,
      'functionResponse': None,
      'executableCode': None,
      'functionCall': None}]},
   'groundingMetadata': None,
   'customMetadata': None,
   'longRunningToolIds': None,
   'errorCode': None,
   'timestamp': 1749406356.673932,
   'author': 'user',
   'errorMessage': None,
   'actions': {'artifactDelta': {},
    'escalate': None,
    'skipSummarization': None,
    'transferToAgent': None,
    'requestedAuthConfigs': {},
    'stateDelta': {}}},
 

In [226]:
for event in remote_app.get_session(user_id = 'u_123', session_id = remote_session['id'])['events']:
    print(event)

{'errorMessage': None, 'invocationId': 'e-677976b1-9315-4f8e-96c0-d31af507916e', 'partial': None, 'branch': None, 'id': '1038108851167035392', 'turnComplete': None, 'usageMetadata': None, 'content': {'role': 'user', 'parts': [{'fileData': None, 'thought': None, 'codeExecutionResult': None, 'inlineData': None, 'text': 'What do you do?', 'videoMetadata': None, 'functionResponse': None, 'thoughtSignature': None, 'executableCode': None, 'functionCall': None}]}, 'groundingMetadata': None, 'customMetadata': None, 'longRunningToolIds': None, 'errorCode': None, 'timestamp': 1749406356.673932, 'author': 'user', 'interrupted': None, 'actions': {'artifactDelta': {}, 'escalate': None, 'skipSummarization': None, 'transferToAgent': None, 'requestedAuthConfigs': {}, 'stateDelta': {}}}
{'errorMessage': None, 'invocationId': 'e-677976b1-9315-4f8e-96c0-d31af507916e', 'partial': None, 'branch': None, 'id': '5411104089343787008', 'turnComplete': None, 'usageMetadata': None, 'content': {'role': 'model', 'p

In [155]:
remote_app.delete_session(user_id = 'u_123', session_id = remote_session['id'])

In [156]:
remote_app.list_sessions(user_id = 'u_123')

{'sessions': []}

In [157]:
delete_app = False

if delete_app and remote_app:
    task = remote_app.delete(force = True)

more
- create session with specified id

---
### TESTING WORK

In [21]:
users = ['u_123', 'gradio_user_123', 'mesop_user_123']
test_user_id = users[2]
test_session_id = ''

In [22]:
sessions = remote_app.list_sessions(user_id = test_user_id)['sessions']
sessions

[{'id': '4795942726765903872',
  'lastUpdateTime': 1749424766.983624,
  'state': {},
  'events': [],
  'appName': '1988805428414251008',
  'userId': 'mesop_user_123'}]

In [19]:
try_session = sessions[0]['id']
for event in remote_app.get_session(user_id = test_user_id, session_id = try_session)['events']:
    print(event)


{'errorMessage': None, 'invocationId': 'e-410a52b7-f842-4943-a801-800882c5d34f', 'partial': None, 'branch': None, 'id': '2041285668163813376', 'turnComplete': None, 'usageMetadata': None, 'content': {'role': 'user', 'parts': [{'fileData': None, 'thought': None, 'codeExecutionResult': None, 'text': 'help me get started', 'inlineData': None, 'videoMetadata': None, 'thoughtSignature': None, 'functionResponse': None, 'executableCode': None, 'functionCall': None}]}, 'groundingMetadata': None, 'customMetadata': None, 'longRunningToolIds': None, 'errorCode': None, 'timestamp': 1749424765.824724, 'author': 'user', 'interrupted': None, 'actions': {'artifactDelta': {}, 'escalate': None, 'skipSummarization': None, 'transferToAgent': None, 'requestedAuthConfigs': {}, 'stateDelta': {}}}
{'interrupted': None, 'invocationId': 'e-410a52b7-f842-4943-a801-800882c5d34f', 'partial': None, 'branch': None, 'id': '2352034042452377600', 'turnComplete': None, 'usageMetadata': None, 'content': {'role': 'model',

In [20]:
event['content']['parts'][-1]['text']

'To get started, I need you to provide a document. You can do this in one of two ways:\n\n1.  **Upload a PDF or PNG file directly.**\n2.  **Provide a GCS URI (bucket and path) to the document.**\n\nPlease provide the document you would like to process using one of these methods.\n'

In [232]:
for sess in remote_app.list_sessions(user_id = test_user_id)['sessions']:
    remote_app.delete_session(user_id = 'gradio_user_123', session_id = sess['id'])

In [23]:
for u in users:
    for s in remote_app.list_sessions(user_id = u)['sessions']:
        remote_app.delete_session(user_id = u, session_id = s['id'])
